In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [16]:
# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.01
NUM_EPOCHS = 30

# Architecture
NUM_CLASSES = 10
BATCH_SIZE = 128
DEVICE = torch.device('cuda:0')
#DEVICE = "cuda:0"
GRAYSCALE = False

In [17]:
train_dataset = datasets.CIFAR10(root='data', 
                                 train=True, 
                                 transform=transforms.ToTensor(),
                                 download=True)

test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)

Files already downloaded and verified


In [18]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes, grayscale):
        self.inplanes = 64
        if grayscale:
            in_dim = 1
        else:
            in_dim = 3
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1, padding=2)
        #self.fc = nn.Linear(2048 * block.expansion, num_classes)
        self.fc = nn.Linear(2048, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2. / n)**.5)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        #x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        logits = self.fc(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas

def resnet101(num_classes, grayscale):
    model = ResNet(block=Bottleneck, 
                   layers=[3, 4, 23, 3],
                   num_classes=NUM_CLASSES,
                   grayscale=grayscale)
    return model

In [19]:
torch.manual_seed(RANDOM_SEED)

model = resnet101(NUM_CLASSES, GRAYSCALE)
model.to(DEVICE)
 
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

In [20]:
import keyboard
import time

def simulate_key_press(key):
    keyboard.press(key)
    time.sleep(0.1)  # Halte die Taste für 0.1 Sekunden gedrückt
    keyboard.release(key)


In [21]:
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
    
time.sleep(1)
simulate_key_press('p')
start_time = time.time()

# use random seed for reproducibility (here batch shuffling)
torch.manual_seed(RANDOM_SEED)

for epoch in range(NUM_EPOCHS):
    
    model.train()
    
    for batch_idx, (features, targets) in enumerate(train_loader):
    
        ### PREPARE MINIBATCH
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
                
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()

    print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d}')
  
total_training_time = (time.time() - start_time)

total_accuracy = compute_accuracy(model, valid_loader, device=DEVICE)
print('Test accuracy: %.2f%%' % total_accuracy)
    
print(f'Total Training Time: {total_training_time:.2f} sec')

time.sleep(1)
simulate_key_press('p')

Epoch: 001/030
Epoch: 002/030
Epoch: 003/030
Epoch: 004/030
Epoch: 005/030
Epoch: 006/030
Epoch: 007/030
Epoch: 008/030
Epoch: 009/030
Epoch: 010/030
Epoch: 011/030
Epoch: 012/030
Epoch: 013/030
Epoch: 014/030
Epoch: 015/030
Epoch: 016/030
Epoch: 017/030
Epoch: 018/030
Epoch: 019/030
Epoch: 020/030
Epoch: 021/030
Epoch: 022/030
Epoch: 023/030
Epoch: 024/030
Epoch: 025/030
Epoch: 026/030
Epoch: 027/030
Epoch: 028/030
Epoch: 029/030
Epoch: 030/030
Test accuracy: 75.41%
Total Training Time: 855.78 sec


In [32]:
import csv
import os

#Zum loggen der Trainingslaufzeit und accuracy  des Models

def add_row_to_csv(filename, datensatz_name, frequency, voltage, runtime, accuracy):
    # Daten für die neue Zeile
    new_row = {'Datensatz': datensatz_name, 'Frequenz': frequency, 'voltage': voltage, 'Laufzeit': runtime, 'Accuracy': accuracy}
    
    # Überprüfe, ob die Datei existiert
    file_exists = os.path.isfile(filename)
    
    # Öffne die CSV-Datei im Modus 'a' (append), um Werte hinzuzufügen
    with open(filename, 'a', newline='') as csvfile:
        # Erstelle einen CSV-Writer, wenn die Datei neu erstellt wird
        writer = csv.DictWriter(csvfile, fieldnames=['Datensatz', 'Frequenz', 'voltage', 'Laufzeit', 'Accuracy'])
        
        # Schreibe die Kopfzeile, falls die Datei neu erstellt wurde
        if not file_exists:
            writer.writeheader()
        
        # Schreibe die neue Zeile in die CSV-Datei
        writer.writerow(new_row)

# Beispielaufruf der Funktion
filename = 'resnet101_cifar_spannung.csv'
datensatz_name = 'cifar'
frequency = '3600'
voltage = '1.1'
runtime = round(total_training_time, 2)
print(total_accuracy)
accuracy = round(total_accuracy.item(), 2)

add_row_to_csv(filename, datensatz_name, frequency, voltage, runtime, accuracy)


tensor(74.3600, device='cuda:0')
